In [1]:
import sys; sys.path.append('../..')
import random
import pyzx as zx
import os
import pickle
import time

In [2]:
import logging

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [3]:
def sliding_window(elements, window_size):
    if len(elements) < window_size:
        return [elements]
    
    for i in range(len(elements) - window_size + 1):
        yield elements[i:i+window_size]

### Heuristic simplification
- When simplifying ZX-diagrams with T-spiders, simplification routines like full_reduce lead to a very high two-qubit gate count.
- When using heuristic-based approaches we can circumvent the problem to some extent leading to better overall circuit cost after optimization

In [4]:
random.seed(1342)
g = zx.generate.cliffordT(qubits=10, depth=500, p_t=0.3, p_cnot=0.5)
c = zx.Circuit.from_graph(g)
print(c.stats())

Circuit  on 10 qubits with 500 gates.
        132 is the T-count
        368 Cliffords among which
        260 2-qubit gates (260 CNOT, 0 other) and
        0 Hadamard gates.


In [5]:
time_copy = time.perf_counter()
g_copy = g.copy()
print(f"Time copy: {time.perf_counter() - time_copy}")

time_clone = time.perf_counter()
g_clone = g.clone()
print(f"Time clone: {time.perf_counter() - time_clone}")

Time copy: 0.002775599976303056
Time clone: 0.0002696999872568995


In [6]:
c = zx.optimize.basic_optimization(c.split_phase_gates())
g = c.to_graph()
g_tele = zx.simplify.teleport_reduce(g)
g_tele.track_phases = False
print(zx.Circuit.from_graph(g).split_phase_gates().stats())

Circuit  on 10 qubits with 464 gates.
        56 is the T-count
        408 Cliffords among which
        252 2-qubit gates (248 CNOT, 4 other) and
        46 Hadamard gates.


In [7]:
g_full = g_tele.copy()
zx.simplify.full_reduce(g_full)
print(zx.extract_circuit(g_full.copy()).stats())

Circuit  on 10 qubits with 439 gates.
        56 is the T-count
        383 Cliffords among which
        287 2-qubit gates (67 CNOT, 220 other) and
        90 Hadamard gates.


In [8]:
applied_matches_la = []

for la in range(0,4):
    print("Lookahead:", la)
    start_time = time.time()
    g_greedy_la = g_tele.copy()
    iterations, applied_matches = zx.simplify.greedy_simp(g_greedy_la, lookahead=la)
    applied_matches_la.append(applied_matches)
    print(zx.extract_circuit(g_greedy_la.copy()).stats())
    print("Time:", time.time()-start_time)
    print("---------------------------------")

INFO:root:greedy_wire_reduce_count: 95


Lookahead: 0
Circuit  on 10 qubits with 593 gates.
        56 is the T-count
        537 Cliffords among which
        232 2-qubit gates (3 CNOT, 229 other) and
        226 Hadamard gates.
Time: 0.09786558151245117
---------------------------------
Lookahead: 1


INFO:root:greedy_wire_reduce_count: 93


Circuit  on 10 qubits with 583 gates.
        56 is the T-count
        527 Cliffords among which
        222 2-qubit gates (2 CNOT, 220 other) and
        227 Hadamard gates.
Time: 2.1193220615386963
---------------------------------
Lookahead: 2


INFO:root:greedy_wire_reduce_count: 116
INFO:root:greedy_wire_reduce_count: 5
INFO:root:greedy_wire_reduce_count: 6


Circuit  on 10 qubits with 531 gates.
        56 is the T-count
        475 Cliffords among which
        233 2-qubit gates (5 CNOT, 228 other) and
        183 Hadamard gates.
Time: 21.167829513549805
---------------------------------
Lookahead: 3


INFO:root:greedy_wire_reduce_count: 143


Circuit  on 10 qubits with 516 gates.
        56 is the T-count
        460 Cliffords among which
        247 2-qubit gates (6 CNOT, 241 other) and
        164 Hadamard gates.
Time: 96.2125186920166
---------------------------------


In [9]:
first_item = applied_matches_la[0]

for index, applied_matches in enumerate(applied_matches_la[1:], start=1):
    differences = [i for i in range(min(len(first_item), len(applied_matches))) if first_item[i] != applied_matches[i]]
    # differences = [i for i, element in enumerate(first_item) if element not in applied_matches] + [i for i, element in enumerate(applied_matches) if element not in first_item]
    if differences:
        first_diff_index = differences[0]
        print("Items in list 0 beginning at " + str(first_diff_index) + " :", first_item[first_diff_index:first_diff_index+index+1])
        print("Items in list " + str(index) + " beginning at " + str(first_diff_index) + " :", applied_matches[first_diff_index:first_diff_index+index+1])
        print("")

Items in list 0 beginning at 26 : [((393, 397), (2, -2)), ((12,), (1.0, [11, 34], -1))]
Items in list 1 beginning at 26 : [((78,), (1.0, [79, 52], -1)), ((79, 90), (3, -2))]

Items in list 0 beginning at 0 : [((694, 695), (5, -2)), ((700, 701), (5, -2)), ((682, 683), (5, -2))]
Items in list 2 beginning at 0 : [((482,), (-5.0, [486, 495, 398, 433, 444], -1)), ((486,), (6.0, [518, 454, 344, 495, 398, 433, 444], -1)), ((337,), (-6.0, [314, 276, 354, 362, 336, 344, 353], -1))]

Items in list 0 beginning at 13 : [((644, 652), (3, -2)), ((457, 473), (2, -2)), ((690, 693), (2, -2)), ((678, 687), (2, -2))]
Items in list 3 beginning at 13 : [((231, 244), (1, -2)), ((204, 255), (3, -2)), ((256,), (1.0, [257, 220], -1)), ((257, 259), (4, -2))]



In [10]:
g_nu = g_tele.copy()
zx.simplify.greedy_simp_neighbors(g_nu)
print(zx.extract_circuit(g_nu.copy()).stats())

Circuit  on 10 qubits with 586 gates.
        56 is the T-count
        530 Cliffords among which
        228 2-qubit gates (2 CNOT, 226 other) and
        221 Hadamard gates.


In [11]:
g_sim_n = g_tele.copy()
zx.simplify.sim_anneal_simp_neighbors(g_sim_n)
print(zx.extract_circuit(g_sim_n.copy()).stats())

final num edges:  480
Circuit  on 10 qubits with 610 gates.
        56 is the T-count
        554 Cliffords among which
        230 2-qubit gates (2 CNOT, 228 other) and
        245 Hadamard gates.


In [12]:
g_sim = g_tele.copy()
zx.simplify.sim_anneal_simp(g_sim)
print(zx.extract_circuit(g_sim.copy()).stats())

TypeError: 'int' object is not subscriptable